#  Сверточные нейронные сети
Реализация стеккинга обучения моделей и метамадели на датасете из игральных карт

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import pandas as pd
from PIL import Image

In [20]:
torch.__version__
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1660 Ti with Max-Q Design'

In [51]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False


img_transforms = transforms.Compose([
    transforms.Resize((64,64)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
    ])


train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./valid/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image) 

batch_size=64

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

for batch_ndx, sample in enumerate(train_data_loader):
    print(sample[0].shape)
    print(batch_ndx)

torch.Size([64, 3, 64, 64])
0
torch.Size([64, 3, 64, 64])
1
torch.Size([64, 3, 64, 64])
2
torch.Size([64, 3, 64, 64])
3
torch.Size([64, 3, 64, 64])
4
torch.Size([64, 3, 64, 64])
5
torch.Size([64, 3, 64, 64])
6
torch.Size([64, 3, 64, 64])
7
torch.Size([64, 3, 64, 64])
8
torch.Size([64, 3, 64, 64])
9
torch.Size([64, 3, 64, 64])
10
torch.Size([64, 3, 64, 64])
11
torch.Size([64, 3, 64, 64])
12
torch.Size([64, 3, 64, 64])
13
torch.Size([64, 3, 64, 64])
14
torch.Size([64, 3, 64, 64])
15
torch.Size([64, 3, 64, 64])
16
torch.Size([64, 3, 64, 64])
17
torch.Size([64, 3, 64, 64])
18
torch.Size([64, 3, 64, 64])
19
torch.Size([64, 3, 64, 64])
20
torch.Size([64, 3, 64, 64])
21
torch.Size([64, 3, 64, 64])
22
torch.Size([64, 3, 64, 64])
23
torch.Size([64, 3, 64, 64])
24
torch.Size([64, 3, 64, 64])
25
torch.Size([64, 3, 64, 64])
26
torch.Size([64, 3, 64, 64])
27
torch.Size([64, 3, 64, 64])
28
torch.Size([64, 3, 64, 64])
29
torch.Size([64, 3, 64, 64])
30
torch.Size([64, 3, 64, 64])
31
torch.Size([64, 3,

In [22]:
file_path_test ="./cards.csv"
data = pd.read_csv(file_path_test, encoding='cp1251', )

In [23]:
len(data["labels"].unique())



53

In [7]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=53):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [34]:
cnnnet = CNNNet()

NameError: name 'CNNNet' is not defined

----

In [11]:
cnnnet(torch.rand(1, 3,224, 224, requires_grad=True))[0].shape

torch.Size([53])

In [20]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"):
    acc = 0
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        if (acc< num_correct / num_examples):
                acc = num_correct / num_examples
                
        if (19< epoch):
                acc = num_correct / num_examples
                torch.save(model,f'model{epoch}_{acc}.pt')        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [52]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

img_transforms = transforms.Compose([
    transforms.Resize((224,224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./valid/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=32
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
device = torch.device("cuda") 


In [ ]:
torch.cuda.is_available()

True

In [15]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

In [16]:
train(cnnnet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=21, device=device)

Epoch: 1, Training Loss: 3.55, Validation Loss: 2.55, accuracy = 0.23
Epoch: 2, Training Loss: 2.47, Validation Loss: 1.89, accuracy = 0.33
Epoch: 3, Training Loss: 2.14, Validation Loss: 1.72, accuracy = 0.39
Epoch: 4, Training Loss: 1.95, Validation Loss: 1.40, accuracy = 0.55
Epoch: 5, Training Loss: 1.78, Validation Loss: 1.37, accuracy = 0.53
Epoch: 6, Training Loss: 1.67, Validation Loss: 1.29, accuracy = 0.55
Epoch: 7, Training Loss: 1.62, Validation Loss: 1.34, accuracy = 0.57
Epoch: 8, Training Loss: 1.50, Validation Loss: 1.12, accuracy = 0.66
Epoch: 9, Training Loss: 1.35, Validation Loss: 1.01, accuracy = 0.74
Epoch: 10, Training Loss: 1.27, Validation Loss: 0.92, accuracy = 0.73


KeyboardInterrupt: 

In [26]:
# Загрузка предварительно обученной модели MobileNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# Перемещение модели на устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Пример загрузки изображения и его преобразования в тензор
input_image = Image.open("./train/ace of clubs/001.jpg")
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Применение преобразований к изображению
input_tensor = preprocess(input_image)

# Добавление размерности батча (batch dimension) для ожидаемого входа модели
input_tensor = input_tensor.unsqueeze(0)

# Передача тензора на устройство (GPU или CPU)
input_tensor = input_tensor.to(device)

# Изменение последнего слоя классификатора модели для адаптации к вашим данным
num_classes = 53
model.classifier[1] = nn.Linear(1280, num_classes)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)




Using cache found in C:\Users\anon/.cache\torch\hub\pytorch_vision_v0.10.0


In [34]:
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, datasets
from PIL import Image
from torch.nn import functional as F

# Определение функции обучения
# Определение функции обучения
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"):
    acc = 0
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Перемещаем модель на устройство
            model.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            # Перемещаем модель на устройство
            model.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets) 
            valid_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        
        if (acc< num_correct / num_examples):
            acc = num_correct / num_examples
                
        if (19< epoch):
            acc = num_correct / num_examples
            torch.save(model, f'model{epoch}_{acc}.pt')        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

batch_size = 16
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
# Функция проверки изображения
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

# Трансформации для изображений
img_transforms = transforms.Compose([
    transforms.Resize((224,224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Пути к данным
train_data_path = "./train/"
val_data_path = "./valid/"

# Создание датасетов
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=img_transforms, is_valid_file=check_image)

# DataLoader для обучения и валидации
batch_size = 10
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка предварительно обученной модели MobileNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# Перемещение модели на устройство (GPU или CPU)
model.to(device)

# Изменение последнего слоя классификатора модели для адаптации к вашим данным
num_classes = 53
model.classifier[1] = nn.Linear(1280, num_classes)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)




Using cache found in C:\Users\anon/.cache\torch\hub\pytorch_vision_v0.10.0


In [43]:
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, datasets
from PIL import Image
from torch.nn import functional as F

# Определение функции обучения
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"):
    acc = 0
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Перемещаем модель на устройство
            model.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            # Перемещаем модель на устройство
            model.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets) 
            valid_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        
        if (acc< num_correct / num_examples):
            acc = num_correct / num_examples
                
        if (19< epoch):
            acc = num_correct / num_examples
            torch.save(model, f'model{epoch}_{acc}.pt')        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

# Функция проверки изображения
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

# Трансформации для изображений
img_transforms = transforms.Compose([
    transforms.Resize((224,224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Пути к данным
train_data_path = "./train/"
val_data_path = "./valid/"

# Создание датасетов
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=img_transforms, is_valid_file=check_image)

# DataLoader для обучения и валидации
batch_size = 10
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка предварительно обученной модели SqueezeNet
model = torchvision.models.squeezenet1_0(pretrained=True)

# Перемещение модели на устройство (GPU или CPU)
model.to(device)

# Изменение последнего слоя классификатора модели для адаптации к вашим данным
num_classes = 53  # ваше количество классов
model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


C:\Users\anon\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to C:\Users\anon/.cache\torch\hub\checkpoints\squeezenet1_0-b66bff10.pth
100%|██████████| 4.78M/4.78M [00:18<00:00, 265kB/s]


In [38]:

print(loss_fn)


NameError: name 'loss_fn' is not defined

In [44]:
train(model, optimizer, criterion, train_data_loader, val_data_loader, epochs=10, device=device)

Epoch: 1, Training Loss: 3.97, Validation Loss: 3.94, accuracy = 0.03
Epoch: 2, Training Loss: 3.87, Validation Loss: 3.88, accuracy = 0.02
Epoch: 3, Training Loss: 3.36, Validation Loss: 2.85, accuracy = 0.18
Epoch: 4, Training Loss: 2.68, Validation Loss: 2.04, accuracy = 0.37
Epoch: 5, Training Loss: 2.21, Validation Loss: 1.64, accuracy = 0.49
Epoch: 6, Training Loss: 1.89, Validation Loss: 1.47, accuracy = 0.54
Epoch: 7, Training Loss: 1.66, Validation Loss: 1.23, accuracy = 0.62
Epoch: 8, Training Loss: 1.49, Validation Loss: 1.02, accuracy = 0.66
Epoch: 9, Training Loss: 1.36, Validation Loss: 0.97, accuracy = 0.70
Epoch: 10, Training Loss: 1.23, Validation Loss: 0.88, accuracy = 0.72


In [50]:
model.eval()
#model.half()
x = torch.rand(1, 3,224, 224, requires_grad=True).to(device)
torch_out = model(x)

In [51]:
# Сохранение модели в формате ONNX
torch.onnx.export(model,                   # Модель
                  torch.randn(1, 3, 224, 224).to(device), # Пример входных данных
                  "squeezenet.onnx",            # Путь для сохранения
                  export_params=True,       # Сохранять ли параметры модели
                  opset_version=11,         # Версия ONNX
                  do_constant_folding=True, # Оптимизировать ли константные узлы
                  input_names=['input'],    # Имена входных узлов
                  output_names=['output'],  # Имена выходных узлов
                  dynamic_axes={'input': {0: 'batch_size'}, # Динамические оси
                                'output': {0: 'batch_size'}})

In [47]:
import onnx
import onnxruntime
import numpy as np

In [52]:
onnx_model = onnx.load("./squeezenet.onnx")

In [53]:


def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()

    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()




из перебранных мной ace of clubs он угадал 4 из 5,интересно что он везде угдал масть но в перой картинке предположи что это король вместо туза

In [54]:
predicted_class = predict_image("./squeezenet.onnx", "./test/ace of clubs/5.jpg", img_transforms, device)
print("Предсказанный класс:", predicted_class)

class_names = train_data.classes
print("Имя класса :",class_names[predicted_class])

Предсказанный класс: 0
Имя класса : ace of clubs


In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import onnxruntime

# Функция для предсказания класса изображения
def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()
    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()

# Путь к вашей модели ONNX
model_path = "mobile.onnx"

# Путь к вашему тестовому набору данных
test_data_path = "./test/"


# Преобразования изображений
img_transforms = transforms.Compose([
    transforms.Resize((224, 224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [42]:


# Инициализация списка для хранения предсказанных классов и истинных меток
predicted_classes = []
true_labels = []

# Пройти по всем изображениям в тестовом наборе данных
for root, dirs, files in os.walk(test_data_path):
    for file in files:
        if file.endswith(".jpg"):  # Проверка, что файл - изображение
            image_path = os.path.join(root, file)
            true_label = os.path.basename(root)  # Извлечение истинного класса из пути
            predicted_class = predict_image(model_path, image_path, img_transforms, device)
            predicted_classes.append(predicted_class)
            true_labels.append(true_label)


In [50]:
untocken_classes = []
for i in predicted_classes:
    untocken_classes.append(class_names[i])


In [25]:
class_names

NameError: name 'class_names' is not defined

In [63]:

# Вычисление метрик
accuracy = accuracy_score(true_labels, untocken_classes)
precision = precision_score(true_labels, untocken_classes, average='weighted')
recall = recall_score(true_labels, untocken_classes, average='weighted')
f1 = f1_score(true_labels, untocken_classes, average='weighted')

# Вывод метрик
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

NameError: name 'accuracy_score' is not defined

Accuracy: 0.6830188679245283
Precision: 0.749387771557583
Recall: 0.6830188679245283
F1 Score: 0.677947628891025 это метрики mobilenet

---------------------------------------


In [57]:
import os
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import onnxruntime

# Функция для предсказания класса изображения для двух моделей
def predict_image(model_paths, image_path, img_transforms, device):
    predicted_classes = []
    for model_path in model_paths:
        # Загрузка ONNX модели
        session = onnxruntime.InferenceSession(model_path)

        # Открытие изображения
        image = Image.open(image_path)

        # Применение преобразований к изображению
        image = img_transforms(image).unsqueeze(0)

        # Получение входного имени и выходного имени из модели
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name

        # Выполнение предсказания
        input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()
        outputs = session.run([output_name], {input_name: input_data})

        # Получение индекса предсказанного класса
        predicted_class = np.argmax(outputs[0], axis=1)
        predicted_classes.append(predicted_class.item())

    return predicted_classes

# Пути к вашим моделям ONNX
model_paths = ["mobile.onnx", "model.onnx","squeezenet.onnx"]

# Пути к вашим данным для предсказания
train_data_path = "./test/"
val_data_path = "./valid/"

# Преобразования изображений
img_transforms = transforms.Compose([
    transforms.Resize((224, 224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Инициализация списка для хранения предсказанных классов и истинных меток
predicted_classes_model1 = []
predicted_classes_model2 = []
predicted_classes_model3 = []
true_labels = []

# Пройти по всем изображениям в тренировочном и валидационном наборах данных
for data_path in [train_data_path, val_data_path]:
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith(".jpg"):  # Проверка, что файл - изображение
                image_path = os.path.join(root, file)
                true_label = os.path.basename(root)  # Извлечение истинного класса из пути
                predicted_classes = predict_image(model_paths, image_path, img_transforms, device)
                predicted_classes_model1.append(predicted_classes[0])
                predicted_classes_model2.append(predicted_classes[1])
                predicted_classes_model3.append(predicted_classes[1])
                true_labels.append(true_label)

# Создание датасета
import pandas as pd

dataset = pd.DataFrame({
    'Model1_Predictions': predicted_classes_model1,
    'Model2_Predictions': predicted_classes_model2,
    'Model3_Predictions': predicted_classes_model3,
    'True_Labels': true_labels
})


In [58]:
dataset

,Model1_Predictions,Model2_Predictions,Model3_Predictions,True_Labels
0,36,21,21,ace of clubs
1,0,0,0,ace of clubs
2,3,0,0,ace of clubs
3,0,0,0,ace of clubs
4,3,0,0,ace of clubs
...,...,...,...,...
525,52,52,52,two of spades
526,45,32,32,two of spades
527,52,52,52,two of spades
528,52,52,52,two of spades


In [59]:
# Обратное преобразование числовых меток в текстовые метки для столбца True_Labels
label_indices = [class_names.index(label) for label in dataset['True_Labels']]


In [60]:
dataset['True_Labels'] = label_indices

In [61]:
dataset

,Model1_Predictions,Model2_Predictions,Model3_Predictions,True_Labels
0,36,21,21,0
1,0,0,0,0
2,3,0,0,0
3,0,0,0,0
4,3,0,0,0
...,...,...,...,...
525,52,52,52,52
526,45,32,32,52
527,52,52,52,52
528,52,52,52,52


In [62]:
dataset.to_csv('dataset.csv', index=False) 

---


In [17]:
df = pd.read_csv('dataset.csv')

In [18]:
from sklearn.model_selection import train_test_split

# Предположим, что df - ваш DataFrame с данными

# Разделение на признаки (X) и целевую переменную (y)
X = df.drop(columns=['True_Labels'])  # Укажите имя столбца с целевой переменной
y = df['True_Labels']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
X_test

,Model1_Predictions,Model2_Predictions,Model3_Predictions
140,28,28,28
398,26,26,26
6,1,3,3
334,13,13,13
322,11,11,11
...,...,...,...
341,15,15,15
431,33,33,33
131,1,26,26
338,14,14,14


In [6]:
y_test

140    28
398    26
6       1
334    13
322    11
       ..
341    15
431    33
131    26
338    14
514    49
Name: True_Labels, Length: 106, dtype: int64

In [21]:
X_train


,Model1_Predictions,Model2_Predictions,Model3_Predictions
137,27,27,27
526,45,32,32
417,30,30,30
517,50,1,1
69,13,13,13
...,...,...,...
71,14,14,14
106,21,21,21
270,1,1,1
435,34,34,34


In [22]:
y_train.head()

137    27
526    52
417    30
517    50
69     13
Name: True_Labels, dtype: int64

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

x_train_date = torch.tensor(X_train.values, dtype=torch.float32)
y_train_date = torch.tensor(y_train.values, dtype=torch.int64)


# Определение архитектуры нейронной сети
class SimpleClassifier(nn.Module):
    def __init__(self):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(3, 64)  # Входной слой: 3 признака -> 64 нейрона
        self.fc2 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 32)  # Скрытый слой: 64 нейрона -> 32 нейрона
        self.fc3 = nn.Linear(32, 53)  # Выходной слой: 32 нейрона -> 53 выход 

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Инициализация модели и функции потерь
model = SimpleClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
epochs = 2000
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(x_train_date)
    loss = criterion(output, y_train_date)
    loss.backward()
    optimizer.step()
    
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/2000], Loss: 6.5281
Epoch [2/2000], Loss: 6.1575
Epoch [3/2000], Loss: 5.8398
Epoch [4/2000], Loss: 5.5683
Epoch [5/2000], Loss: 5.3368
Epoch [6/2000], Loss: 5.1408
Epoch [7/2000], Loss: 4.9730
Epoch [8/2000], Loss: 4.8296
Epoch [9/2000], Loss: 4.7084
Epoch [10/2000], Loss: 4.6032
Epoch [11/2000], Loss: 4.5163
Epoch [12/2000], Loss: 4.4402
Epoch [13/2000], Loss: 4.3743
Epoch [14/2000], Loss: 4.3164
Epoch [15/2000], Loss: 4.2648
Epoch [16/2000], Loss: 4.2184
Epoch [17/2000], Loss: 4.1765
Epoch [18/2000], Loss: 4.1383
Epoch [19/2000], Loss: 4.1053
Epoch [20/2000], Loss: 4.0768
Epoch [21/2000], Loss: 4.0519
Epoch [22/2000], Loss: 4.0305
Epoch [23/2000], Loss: 4.0115
Epoch [24/2000], Loss: 3.9938
Epoch [25/2000], Loss: 3.9777
Epoch [26/2000], Loss: 3.9629
Epoch [27/2000], Loss: 3.9495
Epoch [28/2000], Loss: 3.9374
Epoch [29/2000], Loss: 3.9266
Epoch [30/2000], Loss: 3.9172
Epoch [31/2000], Loss: 3.9088
Epoch [32/2000], Loss: 3.9012
Epoch [33/2000], Loss: 3.8940
Epoch [34/2000], Lo

---

In [26]:
import torch.onnx

# Создание фиктивных входных данных
dummy_input = torch.randn(1, 3)  # Размерность входных данных должна соответствовать вашим данным

# Укажите путь для сохранения модели в формате ONNX
onnx_path = "metamodel.onnx"

# Экспорт модели в формат ONNX
torch.onnx.export(model, dummy_input, onnx_path, verbose=True)

In [6]:
import onnxruntime
import numpy as np

# Создание исполнителя (runtime) для модели ONNX
ort_session = onnxruntime.InferenceSession("./metamodel.onnx")

# Создание входных данных для предсказания (данные должны соответствовать формату, используемому при обучении модели)
input_data = np.array([[10, 20, 30]], dtype=np.float32)
  # Замените ... на ваши входные данные

# Предсказание
outputs = ort_session.run(None, {"onnx::Gemm_0": input_data})




In [10]:
outputs

[array([[-25.41567   , -12.921899  , -84.79633   , -64.49079   ,
         -70.72872   , -29.752794  ,  -3.3705416 ,  15.261076  ,
          -0.90996516,  30.905178  ,  -6.007939  , -84.946754  ,
         -12.913983  , -16.98314   , -22.282791  ,  -7.9085274 ,
         -21.395092  ,  20.0179    ,  27.931566  ,  32.618572  ,
          40.36392   ,  -8.757456  ,  17.811491  , -12.739071  ,
         -12.8525    ,  35.48876   ,  43.322754  ,  13.947449  ,
           6.25141   ,  -8.6482725 ,  45.287987  ,  -7.3162494 ,
          17.354692  ,  42.89952   , -25.465046  ,  32.4281    ,
           5.9982333 , -32.366417  , -29.535975  ,  18.582813  ,
         -23.057295  , -28.449553  , -24.052343  ,   0.9745277 ,
         -21.569006  , -41.7246    , -36.428688  , -52.981213  ,
           5.342592  , -43.894096  , -49.073803  , -47.70679   ,
         -40.245296  ]], dtype=float32)]

In [15]:
x=outputs[0]
x=x[0]

len(x)

53

In [18]:
max_index = np.argmax(x)
print("Индекс максимального числа:", max_index)

Индекс максимального числа: 30


In [27]:
class_names = train_data.classes

In [28]:
class_names

['ace of clubs',
 'ace of diamonds',
 'ace of hearts',
 'ace of spades',
 'eight of clubs',
 'eight of diamonds',
 'eight of hearts',
 'eight of spades',
 'five of clubs',
 'five of diamonds',
 'five of hearts',
 'five of spades',
 'four of clubs',
 'four of diamonds',
 'four of hearts',
 'four of spades',
 'jack of clubs',
 'jack of diamonds',
 'jack of hearts',
 'jack of spades',
 'joker',
 'king of clubs',
 'king of diamonds',
 'king of hearts',
 'king of spades',
 'nine of clubs',
 'nine of diamonds',
 'nine of hearts',
 'nine of spades',
 'queen of clubs',
 'queen of diamonds',
 'queen of hearts',
 'queen of spades',
 'seven of clubs',
 'seven of diamonds',
 'seven of hearts',
 'seven of spades',
 'six of clubs',
 'six of diamonds',
 'six of hearts',
 'six of spades',
 'ten of clubs',
 'ten of diamonds',
 'ten of hearts',
 'ten of spades',
 'three of clubs',
 'three of diamonds',
 'three of hearts',
 'three of spades',
 'two of clubs',
 'two of diamonds',
 'two of hearts',
 'two o

In [29]:
print("Имя класса :",class_names[max_index])

Имя класса : queen of diamonds


----

Главнаая функция с предсказаниями всех моделей и в итоге предсказанием метамодели для итогового результата

In [40]:
def predict_meta(input_data):
    ort_session = onnxruntime.InferenceSession("./metamodel.onnx")

    
    input_data = np.array([[10, 20, 30]], dtype=np.float32)

    outputs = ort_session.run(None, {"onnx::Gemm_0": input_data})
    return outputs

def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()

    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()



In [59]:


pred=[]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models=["mobile.onnx","model.onnx","squeezenet.onnx"]
for i in models:
    predicted_class = predict_image(i, "./test/ace of clubs/5.jpg", img_transforms, device)
    pred.append(predicted_class)

class_names = train_data.classes

predict_meta(pred)
print("Имя класса :",class_names[predicted_class])


Имя класса : ace of clubs


Вывод использование стекинга повышает итоговые метрики модели 